In [10]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tsfresh import select_features
import datetime
from tsfresh import defaults
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.utilities.dataframe_functions import check_for_nans_in_columns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_curve
from sklearn import metrics
import pickle
import joblib
import re
from collections import Counter
import sys, os
import random


### configuration ###

In [11]:
storage_path = r"/storage/users/assafzar/Muscle_Differentiation_AvinoamLab/"
data_csv_path = storage_path + r"data/mastodon/%s%s all detections.csv"
intensity_model_path = storage_path + r"15-12-2022-actin_intensity local dens-False, s%s, s%s train [130, 160] diff window win size 16/track len 30, impute_func-ImputeAllData_impute_zeroes reg MeanOpticalFlowReg_/"
motility_model_path = storage_path + r"15-12-2022-motility local dens-False, s%s, s%s train [130, 160] diff window/track len 30, impute_func-ImputeAllData_impute_zeroes reg MeanOpticalFlowReg_/"
FEATURES_DIR_PATH = f"data/mastodon/features/"
transformed_data_path = '/storage/users/assafzar/Muscle_Differentiation_AvinoamLab/data/mastodon/ts_transformed/%s/ImputeAllData_impute_zeroes/S%s/merged_chunks_reg=MeanOpticalFlowReg_,local_den=False,win size=16.pkl'


REG_METHOD = "MeanOpticalFlowReg_"
IMPUTE_FUNC = "impute_zeroes"
IMPUTE_METHOD = "ImputeAllData"
WIN_SIZE=16
diff_window=[130, 160] 
con_window=[[0, 30], [40, 70], [90, 120], [130, 160], [180, 210], [220, 250]]
transformed_data_path=storage_path + f"data/mastodon/ts_transformed/%s/{IMPUTE_METHOD}_{IMPUTE_FUNC}/S%s/merged_chunks_reg={REG_METHOD},local_den=False,win size={WIN_SIZE}.pkl"
SEGMENT_LEN = 30


In [12]:
def clean_redundant_columns(df):
    remove_cols = []
    cols_to_remove = ["target"]
    for col_to_remove in cols_to_remove:
        for col_name in df.columns:
            if col_name.startswith(col_to_remove):
                remove_cols.append(col_name)
    df = df.drop(columns=remove_cols)
    return df

In [13]:
def downcast_df(data_copy, fillna=True):
    """
    Downcasts the data types of a DataFrame to reduce memory usage.
    :param data_copy: (pd.DataFrame) The DataFrame to be downcasted.
    :param fillna: (bool, optional) Whether to fill NaN values with zero. Default is True.
    :return: (pd.DataFrame) The downcasted DataFrame.
    """
    data_copy = data_copy.copy()
    if fillna:
        data_copy = data_copy.fillna(0)
    data_copy = data_copy.dropna(axis=1)
    cols = list(data_copy.drop(columns="Spot track ID").columns) if "spot track ID" in data_copy.columns else list(
        data_copy.columns)
    for col in cols:
        try:
            if data_copy[col].sum().is_integer():
                data_copy[col] = pd.to_numeric(data_copy[col], downcast='integer')
            else:
                data_copy[col] = pd.to_numeric(data_copy[col], downcast='float')

            if np.isinf(data_copy[col]).sum() > 0:
                data_copy[col] = data_copy[col]
        except:
            continue
    return data_copy


In [14]:
def load_tsfresh_csv(transfromed_pkl_path, modality, vid_num):
    print(f"read data from video number {vid_num}")
    df = pickle.load(open(transfromed_pkl_path % (modality, vid_num), 'rb'))
    df = downcast_df(df, fillna=False)
    df = clean_redundant_columns(df)
    return df

In [15]:
def get_to_run(transformed_data_path, modality, con_train_num=None, con_test_num=None, diff_train_num=None,
               diff_test_num=None):
    diff_df_train, con_df_train, con_df_test, diff_df_test = None, None, None, None

    if diff_train_num:
        diff_df_train = load_tsfresh_csv(transformed_data_path, modality, diff_train_num)
        print(f"diff train len: {diff_df_train.shape}", flush=True)

    if con_train_num:
        con_df_train = load_tsfresh_csv(transformed_data_path, modality, con_train_num)
        print(f"con_df_train len: {con_df_train.shape}", flush=True)

    if con_test_num:
        con_df_test = load_tsfresh_csv(transformed_data_path, modality, con_test_num)

    if diff_test_num:
        diff_df_test = load_tsfresh_csv(transformed_data_path, modality, diff_test_num)

    return diff_df_train, con_df_train, con_df_test, diff_df_test

In [16]:
def get_data(modality, path, con_train_n, diff_train_n, con_test_n, diff_test_n):
    diff_df_train, con_df_train, con_df_test, diff_df_test = get_to_run(transformed_data_path=path,
                                                                            modality=modality,
                                                                            con_train_num=con_train_n,
                                                                            diff_train_num=diff_train_n,
                                                                            con_test_num=con_test_n,
                                                                            diff_test_num=diff_test_n)
    if (con_train_n is not None) and (diff_train_n is not None):
        diff_df = diff_df_train
        con_df = con_df_train

    elif (con_test_n is not None) and (diff_test_n is not None):
        diff_df = diff_df_test
        con_df = con_df_test
    print("ended loading", flush=True)
    return diff_df, con_df

In [17]:
def concat_dfs(diff_df, con_df, diff_t_window=None, con_t_windows=None):
    def set_indexes(df, target, max_val):
        df["Spot track ID"] = df["Spot track ID"] + max_val
        max_val = df["Spot track ID"].max() + 1
        df['target'] = np.array([target for i in range(len(df))])
        return df, max_val

    max_val = 0
    diff_start, diff_end = diff_t_window
    window_size = diff_end - diff_start

    # Erk video
    # Cut the needed time window
    new_diff_df = pd.DataFrame()
    diff_df = diff_df[diff_df["Spot frame"] == diff_end]
    print("size of diff_df: ", diff_df.shape)

    for label, label_df in diff_df.groupby('Spot track ID'):
        # new_diff_df = new_diff_df.append(label_df)
        new_diff_df = pd.concat([new_diff_df, label_df], ignore_index=True)

    # control video
    # Cut the needed time window
    control_df = pd.DataFrame()
    new_label = max(con_df['Spot track ID'].unique()) + 1
    for start, end in con_t_windows:
        tmp_df = con_df[con_df["Spot frame"] == end]
        for label, label_df in tmp_df.groupby('Spot track ID'):
            new_label += 1
            label_df["Spot track ID"] = new_label
            # control_df = control_df.append(label_df)
            control_df = pd.concat([control_df, label_df], ignore_index=True)
    con_df = control_df.copy()
    print("size of con_df: ", con_df.shape)

    new_diff_df, max_val = set_indexes(new_diff_df, target=True, max_val=max_val)
    con_df, _ = set_indexes(con_df, target=False, max_val=max_val)
    total_df = pd.concat([new_diff_df, con_df], ignore_index=True)
    return total_df


In [18]:
def prep_data(diff_df, con_df, diff_t_window, con_t_windows):
    print("\n preparing data", flush=True)
    print("\nconcatenating control data & ERKi data")
    df = concat_dfs(diff_df, con_df, diff_t_window, con_t_windows)
    del diff_df
    del con_df
    df = df.sample(frac=1).reset_index(drop=True)
    print("\nshape after concat_dfs", df.shape)

    df = df.replace([np.inf], np.nan)
    df = df.dropna(axis=1)
    print("\nshape after dropna", df.shape)

    df.index = df['Spot track ID']
    y = pd.Series(df['target'])
    y.index = df['Spot track ID']
    df = df.drop(["target", "Spot frame", "Spot track ID"], axis=1)
    return df, y


In [19]:
def select_features(
    X,
    y,
    test_for_binary_target_binary_feature=defaults.TEST_FOR_BINARY_TARGET_BINARY_FEATURE,
    test_for_binary_target_real_feature=defaults.TEST_FOR_BINARY_TARGET_REAL_FEATURE,
    test_for_real_target_binary_feature=defaults.TEST_FOR_REAL_TARGET_BINARY_FEATURE,
    test_for_real_target_real_feature=defaults.TEST_FOR_REAL_TARGET_REAL_FEATURE,
    fdr_level=defaults.FDR_LEVEL,
    hypotheses_independent=defaults.HYPOTHESES_INDEPENDENT,
    n_jobs=defaults.N_PROCESSES,
    show_warnings=defaults.SHOW_WARNINGS,
    chunksize=defaults.CHUNKSIZE,
    ml_task="auto",
    multiclass=False,
    n_significant=1,
):
    assert isinstance(X, pd.DataFrame), "Please pass features in X as pandas.DataFrame."
    check_for_nans_in_columns(X)
    assert isinstance(y, (pd.Series, np.ndarray)), (
        "The type of target vector y must be one of: " "pandas.Series, numpy.ndarray"
    )
    assert len(y) > 1, "y must contain at least two samples."
    assert len(X) == len(y), "X and y must contain the same number of samples."
    assert (
        len(set(y)) > 1
    ), "Feature selection is only possible if more than 1 label/class is provided"

    if isinstance(y, pd.Series) and set(X.index) != set(y.index):
        raise ValueError("Index of X and y must be identical if provided")

    if isinstance(y, np.ndarray):
        y = pd.Series(y, index=X.index)

    relevance_table = calculate_relevance_table(
        X,
        y,
        ml_task=ml_task,
        multiclass=multiclass,
        n_significant=n_significant,
        n_jobs=n_jobs,
        show_warnings=show_warnings,
        chunksize=chunksize,
        test_for_binary_target_real_feature=test_for_binary_target_real_feature,
        fdr_level=fdr_level,
        hypotheses_independent=hypotheses_independent,
    )

    relevant_features = relevance_table[relevance_table.relevant].feature

    return X.loc[:, relevant_features]


In [20]:
def save_data(dir_path, clf=None, X_train=None, X_test=None, y_train=None, y_test=None):
    """
    Saves a classifier, train & test data of a given model in a directory.
    :param dir_path: (str) Directory path where the files will be saved
    :param clf: (bool, optional) Whether to save the classifier. Default is True.
    :param X_train: (bool, optional) Whether to save the training input data. Default is True.
    :param X_test: (bool, optional) Whether to save the testing input data. Default is True.
    :param y_train: (bool, optional) Whether to save the training target data. Default is True.
    :param y_test: (bool, optional) Whether to save the testing target data. Default is True.
    :return: 
    """
    if X_train is not None:
        X_train.to_csv(dir_path + "/" + "X_train")
    if X_test is not None:
        X_test.to_csv(dir_path + "/" + "X_test")
    if y_test is not None:
        y_test.to_csv(dir_path + "/" + "y_test")
    if y_train is not None:
        y_train.to_csv(dir_path + "/" + "y_train")
    if clf is not None:
        joblib.dump(clf, dir_path + "/" + "clf.joblib")


In [21]:
def evaluate(clf, X_test, y_test):
    pred = clf.predict(X_test)
    # report = classification_report(y_test, pred)

    fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)
    auc_score = metrics.auc(fpr, tpr)
    print(auc_score)
    return auc_score

In [22]:
def evaluate_clf(dir_path, clf, X_test, y_test, y_train, diff_window, con_window):
    auc_score = evaluate(clf, X_test, y_test)
    txt_file = open(dir_path + '/info.txt', 'a')
    txt_file.write(
                   f"\n auc score: {auc_score}"
                   f"\n train samples:{Counter(y_train)}"
                   f"\n {diff_window} ERK, {con_window} con frames"
                   f"\n n features= {X_test.shape}")

    txt_file.close()

    return auc_score

In [23]:
def build_state_prediction_model_light_with_seed(save_dir_path, con_window, diff_window, modality, diff_df_test, con_df_test, diff_df_train, con_df_train, seed, to_save=True):
    """
    duild a classifier with seed number of random forest.
    """
    
    print("loaded data, now start prep data", flush=True)
    X_train, y_train = prep_data(diff_df=diff_df_train, con_df=con_df_train, diff_t_window=diff_window,
                                 con_t_windows=con_window)
    X_train = downcast_df(X_train)

    del diff_df_train
    del con_df_train
    print("deleted diff_df_train, con_df_train", flush=True)

    X_train = select_features(X_train, y_train, n_jobs=10)  # , chunksize=10
    print("Done feature selection", flush=True)

    clf = train_model_with_seed(X_train, y_train, modality, seed)
    if to_save:
        save_data(save_dir_path, X_train=X_train)

    X_test, y_test = prep_data(diff_df=diff_df_test, con_df=con_df_test, diff_t_window=diff_window,
                               con_t_windows=con_window)
    X_test = X_test[list(clf.feature_names_in_)]
    evaluate_clf(save_dir_path, clf, X_test, y_test, y_train, diff_window, con_window)
    
    if to_save:
        save_data(save_dir_path, y_train=y_train, X_test=X_test, y_test=y_test, clf=clf)
    save_data(save_dir_path,clf=clf)
    return clf

In [24]:
def calc_state_trajectory(transformed_tracks_df, clf, n_frames=260):
    df_score = pd.DataFrame(columns=[i for i in range(n_frames)])
    for track_id, track in transformed_tracks_df.groupby("Spot track ID"):
        spot_frames = list(track.sort_values("Spot frame")["Spot frame"])
        diff_score = {"Spot track ID": track_id}
        try:
            for t in spot_frames:
                probs = clf.predict_proba(track[track["Spot frame"] == t].drop(["Spot track ID", "Spot frame"], axis=1))
                diff_score[t] = pd.to_numeric(probs[0][1], downcast='float')

            diff_score_df = pd.DataFrame(diff_score, index=[0])
            df_score = pd.concat([df_score, diff_score_df], ignore_index=True, sort=False)
        except Exception as e:
            print(e)
            print(track[track["Spot frame"] == t].drop(["Spot track ID", "Spot frame"], axis=1).size)
    print(df_score.shape)
    return df_score

In [25]:

def train_model_with_seed(X_train, y_train, modality, seed):

    params_dict = {
        "motility": {'max_depth': 12, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': seed},
        "actin_intensity": {'max_depth': 20, 'min_samples_leaf': 1, 'n_estimators': 200, 'random_state': seed}
    }
    params = params_dict.get(modality) if params_dict.get(modality) is not None else {}

    clf = RandomForestClassifier(**params, )
    clf.fit(X_train, y_train)
    return clf

In [26]:

def train_model_without_seed(X_train, y_train, modality):

    params_dict = {"motility": {'max_depth': 12, 'min_samples_leaf': 1, 'n_estimators': 100},
                   "actin_intensity": {'max_depth': 20, 'min_samples_leaf': 1, 'n_estimators': 200}}
    params = params_dict.get(modality) if params_dict.get(modality) is not None else {}

    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)
    return clf

## build random forest models ##

In [27]:
modality='actin_intensity'

### get video data ###

In [28]:
diff_df_te, con_df_te = get_data(modality, transformed_data_path, None, None, 2, 3)
diff_df_tra, con_df_tra = get_data(modality, transformed_data_path, 1, 5, None, None) 

read data from video number 2
read data from video number 3
ended loading
read data from video number 5
diff train len: (47703, 3158)
read data from video number 1
con_df_train len: (16297, 3158)
ended loading


In [32]:
today = datetime.datetime.now().strftime('%d-%m-%Y')
id_list = [16771, 27879, 16991, 16003, 8598]

for con_train_n, diff_train_n, con_test_n, diff_test_n in [(1, 5, 2, 3)]:
    print(f"\n train: con_n-{con_train_n},dif_n-{diff_train_n}; test: con_n-{con_test_n},dif_n-{diff_test_n}")    
    print(f"model_name: {modality}")

    flag_save_data = True
    
    for i in range(0,1):  
        if i != 0:
            flag_save_data = False 
        dir_path = f"{storage_path}/confidence_interval/{today}-{modality} local dens-False, s{con_train_n}, s{diff_train_n} train" \
               + (f" win size {WIN_SIZE}" if modality != "motility" else "")

        second_dir = f"track len {SEGMENT_LEN}, impute_func-{IMPUTE_METHOD}_{IMPUTE_FUNC} reg {REG_METHOD}"

        seed = i + 1  # Seed value starts from 1
        save_dir_path = dir_path + "/" + second_dir + "/" + f"model_num-{i+1}, seed_num-{seed}"


        sec_dir=f'{i}'
        os.makedirs((save_dir_path), exist_ok=True)

        clf =build_state_prediction_model_light_with_seed(save_dir_path=save_dir_path,
                                             con_window=con_window,
                                             diff_window=diff_window, modality=modality, diff_df_test=diff_df_te,
                                             con_df_test=con_df_te, diff_df_train=diff_df_tra, con_df_train=con_df_tra, seed=seed, to_save=flag_save_data)
        cols = list(clf.feature_names_in_)
        cols.extend(["Spot track ID", "Spot frame"])

        print("calc avg prob")

        df = diff_df_te[cols].dropna(axis=1)
        filtered_df = df[df['Spot track ID'].isin(id_list)]
        df_score_dif = calc_state_trajectory(filtered_df, clf, n_frames=260)

        pickle.dump(df_score_dif, open(save_dir_path + "/" + f"df_score_vid_num_S{diff_test_n}.pkl", 'wb'))
            



 train: con_n-1,dif_n-5; test: con_n-2,dif_n-3
model_name: actin_intensity
loaded data, now start prep data

 preparing data

concatenating control data & ERKi data
size of diff_df:  (268, 3158)
size of con_df:  (459, 3158)

shape after concat_dfs (727, 3159)

shape after dropna (727, 3159)


/tmp/ipykernel_115061/2825200806.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['target'] = np.array([target for i in range(len(df))])


deleted diff_df_train, con_df_train
Done feature selection

 preparing data

concatenating control data & ERKi data
size of diff_df:  (577, 3158)
size of con_df:  (180, 3158)

shape after concat_dfs (757, 3159)

shape after dropna (757, 3159)
0.8001684960523782


/tmp/ipykernel_115061/2825200806.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['target'] = np.array([target for i in range(len(df))])


calc avg prob
(5, 261)


In [ ]:
with open('/storage/users/assafzar/Muscle_Differentiation_AvinoamLab/confidence_interval/07-06-2023-actin_intensity local dens-False, s2, s3 train win size 16/track len 30, impute_func-ImputeAllData_impute_zeroes reg MeanOpticalFlowReg_/0df_score_vid_num_S5.pkl', 'rb') as f:
    x = pickle.load(f)